### Data ingestion and exploration

##### Required Python packages

In [7]:
import os
import sys
import pandas as pd

##### Verify System Version and set input data path, database path

In [5]:
# Input data path and database path within container
CSV_PATH = "/home/jovyan/challenge/data/ads_spend.csv"
DB_PATH = "/home/jovyan/challenge/database/warehouse.db"

# Verify directory path and python version
print(f"Working directory: {os.getcwd()}")
print(f"Python version: {sys.version}")

Working directory: /home/jovyan/challenge
Python version: 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]


##### Explore input data (ads_spend.csv)

In [ ]:
# Verify if csv exists
if os.path.exists(CSV_PATH):
    
    # Take first 5 rows into pandas format, just for inspection
    df_sample = pd.read_csv(CSV_PATH, nrows=5)
    display(df_sample)
    
else:
    print("CSV file not found")

# Delete dataframe
del df_sample

,date,platform,account,campaign,country,device,spend,clicks,impressions,conversions
0,2025-01-01,Meta,AcctA,Prospecting,MX,Desktop,1115.94,360,15840,29
1,2025-01-01,Google,AcctA,Brand_Search,CA,Mobile,789.43,566,22640,28
2,2025-01-01,Google,AcctA,Prospecting,BR,Desktop,381.40,133,10241,12
3,2025-01-01,Google,AcctC,Prospecting,US,Desktop,1268.34,891,49005,36
4,2025-01-01,Google,AcctA,Brand_Search,BR,Desktop,1229.70,628,21352,31


In [ ]:
# Verify if csv exists
if os.path.exists(CSV_PATH):
    
    # Full data set info
    df_full = pd.read_csv(CSV_PATH)
    print(f"dataset shape: {df_full.shape}")
    print(f"Columns: {list(df_full.columns)}")
    print(f"Spend total: ${df_full['spend'].sum():,.2f}")
    print(f"Date range: {df_full['date'].min()} a {df_full['date'].max()}")
    print(f"Platforms: {df_full['platform'].unique()}")
    print(f"Unique accounts: {df_full['account'].nunique()}")
    
    # Verify data types
    for col, dtype in df_full.dtypes.items():
        print(f"  {col}: {dtype}")
        
else:
    print("CSV file not found")

# Delete dataframe
del df_full